#VLM Benchmark for Neuroradiology

In [ ]:
# Install required libraries
!pip install -q together gspread pandas gspread_dataframe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 1.7 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import random

In [ ]:
# Load data from CSV
data_path = '/content/drive/MyDrive/NeurovisionLab/Neuro_VLM/data.csv'
data = pd.read_csv(data_path)

# Prepare image directory
image_dir = "/content/drive/MyDrive/NeurovisionLab/Neuro_VLM/merged_images"
data

,ID,Age,Sex,Diagnosis,Presentation,Description,Modality,Difficulty
0,A001,78,male,Stroke with haemorrhagic transformation,Dysarthia followed by right upper and lower li...,Left frontal subacute infarction that exhibits...,MRI,easy
1,A002,83,female,Stroke left MCA,Right-sided weakness and aphasia 5 hours ago,Contrast accentuates the loss of grey-white ma...,CT,easy
2,A003,71,male,Stroke - Artery of Percheron territory,Altered sensorium and bilateral lower limb wea...,There are patchy areas of diffusion restrictio...,MRI,easy
3,A004,46,female,Venous hemorrhagic infarct due to cortical vei...,Headache,Acute venous hemorrhagic infarct in the right ...,MRI,easy
4,A005,86,male,Stroke right MCA,Left sided weakness (hemiparesis),Hyperdense right MCA.,CT,easy
...,...,...,...,...,...,...,...,...
95,A096,30,male,Sarcoidosis of the spinal cord,2 month history of neck pain with paresthesia....,"On soft tissue windows, marked mediastinal and...","MRI, CT",average
96,A097,45,female,Neuromyelitis optica (NMO),Back pain and urinary incontinence. Lumbar spi...,The cord demonstrates a longitudinaly extensiv...,MRI,average
97,A098,34,male,Spinal cord infarct,Explosion leading to an extensive laceration t...,The cord demonstrates a long segment of increa...,MRI,easy
98,A099,4,male,Guillain-Barré syndrome,3-day history of leg weakness. Now acutely wor...,The roots of the cauda equina (both dorsal and...,MRI,easy


# GEMINI

In [ ]:
pip install google-generativeai httpx pandas tqdm pillow


In [ ]:
#from pydantic import BaseModel
from openai import OpenAI
import pandas as pd
import os
import httpx
import base64
#import google.generativeai as genai
from PIL import Image
import PIL.Image
import json
from tqdm import tqdm # Import tqdm

In [ ]:
# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "xxx"
# Initialize OpenAI client
client = OpenAI()


import google.generativeai as genai
# Gemini API
genai.configure(api_key="xxx")
model = genai.GenerativeModel(model_name="gemini-1.5-flash")



In [ ]:
import PIL.Image
import os
import pandas as pd
from tqdm import tqdm
import google.generativeai as genai

# Paths to data and images
data_path = "/content/drive/MyDrive/NeurovisionLab/Neuro_VLM/data.csv"
image_dir = "/content/drive/MyDrive/NeurovisionLab/Neuro_VLM/merged_images"

# Load data
data = pd.read_csv(data_path)


# Prepare outputs
outputs = []

# Process each case with progress bar
for _, row in tqdm(data.iterrows(), total=len(data), desc="Processing Cases"):
    patient_id = row["ID"]
    image_path = os.path.join(image_dir, f"{patient_id}.jpg")  # Assuming images are in .jpg format

    if os.path.exists(image_path):
        # Open the image
        sample_image = PIL.Image.open(image_path)

        # Create the prompt (include patient details for context)
        prompt = f"""
        You are an expert neuroradiologist. Analyze the medical image provided and determine the most probable diagnosis.

        ### Case Details:
        - **Patient ID**: {row['ID']}
        - **Sex**: {row['Sex']}
        - **Age**: {row['Age']}
        - **Presentation**: {row['Presentation']}
        """

        # Send prompt and image to Gemini
        response = model.generate_content([prompt, sample_image])  # Pass image directly

        # Extract the most probable diagnosis (assuming it's the entire response text)
        most_probable_diagnosis = response.text.strip()

        # Append to outputs
        outputs.append({
            "Patient ID": row["ID"],
            "Most Probable Diagnosis": most_probable_diagnosis
        })
    else:
        print(f"Image for Patient ID {patient_id} not found.")

# Create a DataFrame for results
output_df = pd.DataFrame(outputs)

# Save the results to a CSV file
output_csv_path = "/content/drive/MyDrive/NeurovisionLab/Neuro_VLM/gemini_most_probable_diagnosis.csv"
output_df.to_csv(output_csv_path, index=False)

print(f"Results saved to {output_csv_path}")

In [ ]:
df

,Patient ID,Explanation,Diagnosis
0,A001,"The MRI images demonstrate a large, well-circu...",large left parietal lobe stroke (most likely a...
1,A002,The CT scan shows a large area of hypodensity ...,acute ischemic stroke in the left middle cereb...
2,A003,"The axial FLAIR MRI image shows bilateral, sym...",thalamic infarction
3,A004,"The MRI images reveal a well-circumscribed, so...",meningioma
4,A005,The CT scan shows a large area of hyperdensity...,chronic subdural hematoma (SDH)
...,...,...,...
95,A096,Based on the provided images and patient histo...,Hodgkin Lymphoma
96,A097,Based on the provided MRI images and clinical ...,tethered spinal cord
97,A098,The MRI images show evidence of significant sp...,Intramedullary edema and hemorrhage:
98,A099,The MRI images demonstrate a significant intra...,ependymoma


#GROK

In [ ]:
import os
import pandas as pd
import base64
from openai import OpenAI
from tqdm import tqdm

# Set your XAI API key
XAI_API_KEY = "xxxx"  # Replace with your actual API key if not using env variable

# Initialize OpenAI client for Grok
client = OpenAI(
    api_key=XAI_API_KEY,
    base_url="https://api.x.ai/v1",
)
# Paths to the data and images
data_path = "/content/drive/MyDrive/NeurovisionLab/Neuro_VLM/data.csv"
image_dir = "/content/drive/MyDrive/NeurovisionLab/Neuro_VLM/merged_images"
output_csv_path = "/content/drive/MyDrive/NeurovisionLab/Neuro_VLM/grok_diagnoses.csv"

# Load the dataset and the existing results
data = pd.read_csv(data_path)
output_df = pd.read_csv(output_csv_path)

# Filter patients starting from A061
remaining_data = data[data["ID"] >= "A061"]

# Helper function to encode images to Base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
    return encoded_string

# Process patients starting from A061
for _, row in tqdm(remaining_data.iterrows(), total=len(remaining_data), desc="Processing Patients from A061"):
    patient_id = row["ID"]
    image_path = os.path.join(image_dir, f"{patient_id}.jpg")  # Assuming images are .jpg format

    if os.path.exists(image_path):
        # Encode the image in Base64
        base64_image = encode_image(image_path)

        # Define the messages for Grok
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                            "detail": "high",
                        },
                    },
                    {
                        "type": "text",
                        "text": "What is the most probable diagnosis for this image?",
                    },
                ],
            },
        ]

        try:
            # Call the Grok model
            completion = client.chat.completions.create(
                model="grok-2-vision-1212",
                messages=messages,
                stream=True,
                temperature=0.01,
            )

            # Collect the response
            response_text = ""
            for chunk in completion:
                response_text += chunk.choices[0].delta.content or ""

            # Print and update the result
            print(f"\nResponse for Patient ID {patient_id}: {response_text.strip()}")
            output_df.loc[output_df["Patient ID"] == patient_id, "Response"] = response_text.strip()

        except Exception as e:
            print(f"Error processing Patient ID {patient_id}: {e}")
            output_df.loc[output_df["Patient ID"] == patient_id, "Response"] = f"Error: {e}"
    else:
        print(f"Image for Patient ID {patient_id} not found.")
        output_df.loc[output_df["Patient ID"] == patient_id, "Response"] = "Image not found"

# Save the updated DataFrame back to the same CSV
output_df.to_csv(output_csv_path, index=False)

print(f"\nResults updated and saved to {output_csv_path}")

# OPENAI

In [ ]:
#OPENAI

# Paths to data and images
data_path = "/content/drive/MyDrive/NeurovisionLab/Neuro_VLM/data.csv"
image_dir = "/content/drive/MyDrive/NeurovisionLab/Neuro_VLM/merged_images"

# Load CSV data
data = pd.read_csv(data_path)

# Define the expected output format using Pydantic
class DiagnosisOutput(BaseModel):
    modality: str
    most_probable_diagnosis: str
    differential_diagnosis_1: str
    differential_diagnosis_2: str
    interpretation: str

# Prepare outputs
outputs = []

# Process each case
for _, row in data.iterrows():
    patient_id = row["ID"]
    image_path = os.path.join(image_dir, f"{patient_id}.jpg")  # Assuming .jpg format for all images

    if os.path.exists(image_path):
        # Create the prompt
        prompt = f"""
        You are an expert neuroradiologist. Please analyze the provided image along with the case details below and provide the following:

        1. **Modality and Sequence**: Specify the imaging modality (e.g., MRI, CT) and sequence (e.g., FLAIR, T2-weighted) if applicable.
        2. **Most Probable Diagnosis**: Identify the most likely diagnosis based on the imaging findings and case details.
        3. **Two Differential Diagnoses**: Provide two other differential diagnoses that should be considered.
        4. **Interpretation**: Summarize the key imaging findings and their significance in 2-5 sentences.

        ### Case Details:
        - **Patient ID**: {row['ID']}
        - **Sex**: {row['Sex']}
        - **Age**: {row['Age']}
        - **Presentation**: {row['Presentation']}
        """

        # Use OpenAI API to generate a response
        completion = client.beta.chat.completions.parse(
            model="o1-mini",
            messages=[
                #{"role": "system", "content": "Provide the following outputs: modality, most probable diagnosis, two differential diagnoses, and interpretation."},
                {"role": "user", "content": prompt},
            ],
            response_format=DiagnosisOutput,
        )

        # Extract and structure the output
        output = {
            "Patient ID": row["ID"],
            "Modality": completion.choices[0].message.parsed.modality,
            "Most Probable Diagnosis": completion.choices[0].message.parsed.most_probable_diagnosis,
            "Differential Diagnosis 1": completion.choices[0].message.parsed.differential_diagnosis_1,
            "Differential Diagnosis 2": completion.choices[0].message.parsed.differential_diagnosis_2,
            "Interpretation": completion.choices[0].message.parsed.interpretation,
        }
        outputs.append(output)
    else:
        print(f"Image for Patient ID {patient_id} not found.")

# Convert the outputs to a DataFrame
output_df = pd.DataFrame(outputs)

# Save the DataFrame to a CSV file
output_csv_path = "/content/drive/MyDrive/NeurovisionLab/Neuro_VLM/results/o1.csv"
output_df.to_csv(output_csv_path, index=False)

print(f"Results exported to {output_csv_path}")


In [ ]:
output_df

,Patient ID,Modality,Most Probable Diagnosis,Differential Diagnosis 1,Differential Diagnosis 2,Interpretation
0,A001,MRI - T2-weighted,Acute ischemic stroke,Primary central nervous system lymphoma,Glioblastoma multiforme,The MRI T2-weighted sequence likely shows hype...
1,A002,CT,Acute ischemic stroke,Intracerebral hemorrhage,Brain tumor,The non-contrast CT scan demonstrates a hyperd...
2,A003,"MRI, T2-weighted",Normal Pressure Hydrocephalus,Chronic Subdural Hematoma,Multi-Infarct Dementia,The imaging shows enlarged ventricles with a n...
3,A004,"MRI, T1-weighted with contrast",Meningioma,Brain metastasis,Glioblastoma multiforme,The imaging reveals an extra-axial mass with d...
4,A005,CT,Acute ischemic stroke,Chronic small vessel ischemic disease,Intracerebral hemorrhage,The non-contrast head CT shows a hypodense are...
...,...,...,...,...,...,...
95,A096,"MRI, T1-weighted with contrast",Spinal Lymphoma,Metastatic spinal disease,"Infectious spondylitis (e.g., tuberculosis)",The imaging reveals an enhancing lesion in the...
96,A097,"MRI, T2-weighted",Conus medullaris syndrome due to spinal cord t...,Transverse myelitis,Spinal cord ischemia,The MRI of the lumbar spine reveals an abnorma...
97,A098,"MRI, T2-weighted",Spinal cord contusion,Spinal cord ischemia,Hematomyelia,The T2-weighted MRI reveals hyperintensity wit...
98,A099,"MRI, T2-weighted",Acute Transverse Myelitis,Guillain-Barré Syndrome,Spinal Cord Tumor,The MRI T2-weighted images likely show hyperin...
